In [7]:
# Loading the required libraries
# Modules used : Pandas, Scikit-Learn, Numpy
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [8]:
# Loading the training data set
df = pd.read_csv("bpp_training_data.csv")
df # overview of the data set

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfYear
0,1646770,9500.0,1126363,8434,132,17,1974,17,0,4593,...,0,1,8,6,0,0,1989,1,17,17
1,1821514,14000.0,1194089,10150,132,31,1980,31,0,1820,...,0,0,0,0,4,2,1989,1,31,31
2,1505138,50000.0,1473654,4139,132,31,1978,31,0,2348,...,0,1,8,6,0,0,1989,1,31,31
3,1671174,16000.0,1327630,8591,132,31,1980,31,0,1819,...,0,0,0,0,4,2,1989,1,31,31
4,1329056,22000.0,1336053,4089,132,31,1984,31,0,2119,...,0,1,6,5,0,0,1989,1,31,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,2192279,16500.0,1701366,6272,136,78,1000,78,0,4215,...,0,0,0,0,0,0,2009,3,19,78
299996,2192338,17000.0,1787662,12919,136,78,1000,78,0,4216,...,0,0,0,0,0,0,2009,3,19,78
299997,2433168,11500.0,1714162,9580,136,78,2005,78,0,4514,...,0,0,0,0,0,0,2009,3,19,78
299998,1586638,32000.0,1386512,11398,132,78,2005,78,3,1202,...,0,1,6,6,0,0,2009,3,19,78


In [9]:
# Removing potentially unwanted features
del df['SalesID']
del df['MachineID']
del df['ModelID']
del df['datasource']
del df['auctioneerID']
df

,SalePrice,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,...,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfYear
0,9500.0,1974,17,0,4593,1744,0,0,0,4,...,0,1,8,6,0,0,1989,1,17,17
1,14000.0,1980,31,0,1820,559,0,0,0,0,...,0,0,0,0,4,2,1989,1,31,31
2,50000.0,1978,31,0,2348,713,59,0,0,2,...,0,1,8,6,0,0,1989,1,31,31
3,16000.0,1980,31,0,1819,558,0,0,0,0,...,0,0,0,0,4,2,1989,1,31,31
4,22000.0,1984,31,0,2119,683,21,0,0,0,...,0,1,6,5,0,0,1989,1,31,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,16500.0,1000,78,0,4215,1500,0,0,0,0,...,0,0,0,0,0,0,2009,3,19,78
299996,17000.0,1000,78,0,4216,1501,0,0,0,0,...,0,0,0,0,0,0,2009,3,19,78
299997,11500.0,2005,78,0,4514,1684,0,0,0,0,...,0,0,0,0,0,0,2009,3,19,78
299998,32000.0,2005,78,3,1202,354,75,0,0,0,...,0,1,6,6,0,0,2009,3,19,78


In [10]:
# Creating a new DataFrame with only the target variable
df_y = df[['SalePrice']]

# Deleting the target variable column from the original DataFrame
del df['SalePrice']


In [11]:
# Creating a linear regression object
reg = linear_model.LinearRegression()

In [12]:
# Splitting 20% of the Data into test set. Will be used later to evaluate the model
x_train, x_test, y_train, y_test = train_test_split(df, df_y, test_size=0.2, random_state=4)

In [18]:
x_train

,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,...,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfYear
152739,1984,303,0,1768,535,43,0,0,4,63,...,0,0,0,0,4,2,2003,10,30,303
2565,1978,164,0,4733,1833,0,0,0,1,72,...,0,0,0,0,4,2,1989,6,13,164
77314,1995,83,0,2611,818,0,23,0,6,8,...,1,0,0,0,0,0,1999,3,24,83
81277,1000,170,0,2060,685,109,0,0,0,51,...,0,1,6,6,0,0,1999,6,19,170
235151,1994,221,1,3138,1055,57,0,0,0,60,...,0,0,0,0,4,2,2007,8,9,221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107578,1977,65,0,1281,375,21,0,0,6,11,...,1,0,0,0,0,0,2001,3,6,65
94601,1987,160,0,496,176,31,0,0,0,2,...,0,0,0,0,0,0,2000,6,8,160
115144,1991,216,0,2462,752,21,0,0,6,8,...,3,0,0,0,0,0,2001,8,4,216
129384,1996,142,0,3507,1244,0,0,0,0,42,...,0,0,0,0,0,0,2002,5,22,142


In [19]:
x_test

,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,...,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfYear
66527,1994,147,0,1372,400,21,0,0,0,38,...,0,0,0,0,0,0,1998,5,27,147
3358,1983,255,0,1268,372,0,0,0,0,5,...,0,0,0,0,0,0,1989,9,12,255
251907,1999,42,3,3864,1397,155,103,9,3,14,...,1,0,0,0,0,0,2008,2,11,42
198370,1999,55,0,4549,1712,0,0,0,5,13,...,1,0,0,0,0,0,2006,2,24,55
228263,2000,137,1,1993,678,20,34,0,2,52,...,0,1,9,6,0,0,2007,5,17,137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111032,1994,117,0,1578,472,59,0,0,0,54,...,0,1,6,6,0,0,2001,4,27,117
243301,1998,311,0,498,176,50,0,0,0,2,...,0,0,0,0,0,0,2007,11,7,311
86898,1990,345,0,2068,685,109,61,26,0,51,...,0,1,5,6,0,0,1999,12,11,345
121027,1989,342,0,185,63,0,0,0,0,39,...,0,0,0,0,0,0,2001,12,8,342


In [20]:
y_test

,SalePrice
66527,9500.0
3358,15000.0
251907,44500.0
198370,11500.0
228263,96000.0
...,...
111032,28500.0
243301,21500.0
86898,13000.0
121027,7250.0


In [21]:
y_train

,SalePrice
152739,39000.0
2565,12500.0
77314,27000.0
81277,17500.0
235151,27500.0
...,...
107578,7500.0
94601,19000.0
115144,23500.0
129384,8500.0


In [13]:
# Fitting the linear model to the training data
reg.fit(x_train, y_train)

LinearRegression()

In [15]:
# Making predictions over the test set
y_pred = reg.predict(x_test)
y_pred

array([[11508.92],
       [14966.92],
       [59621.17],
       ...,
       [38894.92],
       [ 9926.92],
       [19718.92]])

In [16]:
# Calculating the mean square error
print(np.mean((y_pred - y_test)**2))

SalePrice    3.144325e+08
dtype: float64


C:\Users\medas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [17]:
# Evaluating our model on the test set to know its approximate efficiency
reg.score(x_test, y_test)

0.406004196135199